In [1]:
# ChEMBL Database Download / ChEMBL数据库下载地址:
# https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/
# chembl_36_sqlite.tar.gz

# Import required libraries / 导入所需库
import pandas as pd
import sqlite3
from rdkit import Chem

def annotate_with_chembl_ic50_and_targets(df, smiles_column="smiles", db_path="../chembl_36.db"):
    """
    Annotate DataFrame with IC50, main target info, and all known targets for each SMILES.
    为每个SMILES添加IC50、主要靶点信息和所有已知靶点的注释。
    """
    ic50_values = []
    target_names = []
    target_ids = []
    all_targets = []
    
    # Connect to local ChEMBL database / 连接本地ChEMBL数据库
    conn = sqlite3.connect(db_path)
    
    for smiles in df[smiles_column]:
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                print(f"Warning / 警告: Cannot parse SMILES / 无法解析SMILES: {smiles}")
                ic50_values.append(None)
                target_names.append(None)
                target_ids.append(None)
                all_targets.append(None)
                continue
                
            canonical_smiles = Chem.MolToSmiles(mol, canonical=True)
            
            # Query molecule by canonical SMILES / 通过规范化SMILES查询分子
            molecule_query = """
            SELECT md.molregno, md.chembl_id 
            FROM molecule_dictionary md
            JOIN compound_structures cs ON md.molregno = cs.molregno
            WHERE cs.canonical_smiles = ?
            LIMIT 1
            """
            
            molecule_result = pd.read_sql_query(molecule_query, conn, params=[canonical_smiles])
            
            if not molecule_result.empty:
                # Convert to Python native int type / 转换为Python原生int类型
                molregno = int(molecule_result.iloc[0]['molregno'])
                chembl_id = molecule_result.iloc[0]['chembl_id']
                
                # Query activities for this molecule (limit to 20 like original code) / 查询该分子的活性数据（限制20条，与原代码一致）
                activities_query = """
                SELECT a.activity_id, a.standard_type, a.standard_value, a.standard_units, 
                       a.assay_id, ass.tid, td.chembl_id as target_chembl_id, td.pref_name
                FROM activities a
                JOIN assays ass ON a.assay_id = ass.assay_id
                JOIN target_dictionary td ON ass.tid = td.tid
                WHERE a.molregno = ?
                LIMIT 20
                """
                
                activities_result = pd.read_sql_query(activities_query, conn, params=[molregno])
                
                found_ic50 = False
                targets_set = set()
                
                for _, activity in activities_result.iterrows():
                    target_name = activity['pref_name']
                    target_chembl_id = activity['target_chembl_id']
                    
                    if pd.notna(target_name):
                        targets_set.add(str(target_name))
                    
                    # Look for IC50 value (first one found) / 查找IC50值（找到的第一个）
                    if not found_ic50 and activity['standard_type'] == 'IC50' and pd.notna(activity['standard_value']):
                        ic50 = activity['standard_value']
                        units = activity['standard_units'] if pd.notna(activity['standard_units']) else ''
                        ic50_values.append(f"{ic50} {units}")
                        target_names.append(str(target_name) if pd.notna(target_name) else None)
                        target_ids.append(str(target_chembl_id) if pd.notna(target_chembl_id) else None)
                        found_ic50 = True
                
                if not found_ic50:
                    ic50_values.append(None)
                    target_names.append(None)
                    target_ids.append(None)
                
                # Add all known targets as a semicolon-separated string / 将所有已知靶点添加为分号分隔的字符串
                all_targets.append("; ".join(sorted(targets_set)) if targets_set else None)
            else:
                ic50_values.append(None)
                target_names.append(None)
                target_ids.append(None)
                all_targets.append(None)
                
        except Exception as e:
            print(f"Error processing SMILES / 处理SMILES时出错 {smiles}: {str(e)}")
            ic50_values.append(None)
            target_names.append(None)
            target_ids.append(None)
            all_targets.append(None)
    
    conn.close()
    
    df['IC50'] = ic50_values
    df['target_name'] = target_names
    df['target_id'] = target_ids
    df['all_known_targets'] = all_targets
    return df

# Load input data / 加载输入数据
df = pd.read_csv("boltz_results_combined.csv")

# Annotate with ChEMBL data / 使用ChEMBL数据进行注释
df = annotate_with_chembl_ic50_and_targets(df, smiles_column="smiles")

# Save annotated results / 保存注释结果
df.to_csv("Combined_models_with_ChemBL.csv", index=False)